-- Welcome to your new notebook
-- Type here in the cell editor to add code!


In [14]:
%%pyspark
print("Spark started")


StatementMeta(, 02f44c0e-6398-4371-bb76-b1f80ca6f9f6, 17, Finished, Available, Finished, False)

Spark started


### Extracting Dimension Tables 

1. Patients
2. Providers
3. Dates

In [5]:
DROP TABLE IF EXISTS gold.dim_patients;

CREATE TABLE gold.dim_patients
USING DELTA
AS
SELECT
    ROW_NUMBER() OVER (ORDER BY patient_id) AS patient_key,
    patient_id,
    Gender,
    CURRENT_TIMESTAMP() AS load_timestamp,
    FLOOR(DATEDIFF(current_date(), dob) / 365) AS age,
    CASE
        WHEN FLOOR(DATEDIFF(current_date(), dob) / 365) < 18 THEN '0-17'
        WHEN FLOOR(DATEDIFF(current_date(), dob) / 365) BETWEEN 18 AND 35 THEN '18-35'
        WHEN FLOOR(DATEDIFF(current_date(), dob) / 365) BETWEEN 36 AND 55 THEN '36-55'
        ELSE '56+'
    END AS age_group
FROM dbo.patients;










StatementMeta(, 000da8a1-934c-4933-8f2f-6a6dbb4d2476, 8, Finished, Available, Finished, True)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [8]:
create or replace table gold.dim_Providers as
select 
row_number() over(order by provider_id) as provider_key,
provider_id, provider_name,specialty,city,state
from dbo.Providers

StatementMeta(, 02f44c0e-6398-4371-bb76-b1f80ca6f9f6, 10, Finished, Available, Finished, False)

<Spark SQL result set with 0 rows and 0 fields>

In [9]:
create or replace table gold.dim_Dates as
select distinct
    cast(date_format(d,'yyyyMMdd') as int) as date_key,
    d as full_date,
    Year(d) as year,
    Month(d) as month,
    Day(d)as day,
    concat('Q',Quarter(d)) as quarter,
    date_format(d,'EEEE') AS weekday
    from
    (select service_date as d from dbo.Claims
    union
    select claim_date as d from dbo.claims
    union
    select payment_date as d from dbo.Payments
    );


StatementMeta(, 02f44c0e-6398-4371-bb76-b1f80ca6f9f6, 11, Finished, Available, Finished, False)

<Spark SQL result set with 0 rows and 0 fields>

### Extracting Fact Tables

1.Claim
2.Fact Denials
3.Payment

In [7]:
create or replace table gold.fact_claim as
select 
c.claim_id,
p.patient_key,
pr.provider_key,
d.date_key as service_date_key,
c.claim_amount,
DATEDIFF(day, c.service_date, c.claim_date) AS days_to_submit,
CURRENT_TIMESTAMP() AS load_timestamp,
case
when c.claim_status = 'Denied' then 1
else 0
end as Is_Denied
from dbo.claims c
join gold.dim_patients p on c.patient_id = p.patient_id
join gold.dim_providers pr on c.provider_id = pr.provider_id
join gold.dim_dates d on c.service_date = d.full_date;

StatementMeta(, 000da8a1-934c-4933-8f2f-6a6dbb4d2476, 10, Finished, Available, Finished, False)

<Spark SQL result set with 0 rows and 0 fields>

In [2]:
Create or replace table gold.fact_denials as
select
cd.denial_id,
fc.claim_id,
cd.denial_reason,
fc.patient_key,
fc.provider_key,
d.date_key as denial_date_key
from dbo.Claim_denials cd join
gold.fact_claim fc on cd.claim_id = fc.claim_id
join gold.dim_dates d on cd.denial_date = d.full_date;

StatementMeta(, 000da8a1-934c-4933-8f2f-6a6dbb4d2476, 3, Finished, Available, Finished, False)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
create or replace table gold.fact_payment as
select 
pmt.payment_id,
fc.claim_id,
fc.patient_key,
fc.provider_key,
d.date_key as paymentdatekey,
pmt.paid_amount
from dbo.Payments pmt 
join gold.fact_claim fc on pmt.claim_id = fc.claim_id
join gold.dim_dates d on pmt.payment_date = d.full_date;

StatementMeta(, 000da8a1-934c-4933-8f2f-6a6dbb4d2476, 4, Finished, Available, Finished, False)

<Spark SQL result set with 0 rows and 0 fields>